In [111]:
import pandas as pd
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [112]:
cancer_df = pd.read_csv("../data/cancer_reg_cleaned.csv", index_col=0)

cancer_df = cancer_df.select_dtypes(include='number')
cancer_df


,avgAnnCount,avgDeathsPerYear,TARGET_deathRate,incidenceRate,medIncome,popEst2015,povertyPercent,studyPerCap,MedianAge,MedianAgeMale,...,PctPrivateCoverageAlone,PctEmpPrivCoverage,PctPublicCoverage,PctPublicCoverageAlone,PctWhite,PctBlack,PctAsian,PctOtherRace,PctMarriedHouseholds,BirthRate
1,173.000000,70,161.3,411.600000,48127,43269,18.6,23.111234,33.0,32.2,...,53.8,43.6,31.1,15.3,89.228509,0.969102,2.246233,3.741352,45.372500,4.333096
3,427.000000,202,194.8,430.400000,44243,75882,17.1,342.637253,42.8,42.2,...,40.3,35.0,45.3,25.0,91.744686,0.782626,1.161359,1.362643,51.021514,4.603841
4,57.000000,26,144.4,350.100000,49955,10321,12.5,0.000000,48.3,47.8,...,43.9,35.1,44.0,22.7,94.104024,0.270192,0.665830,0.492135,54.027460,6.796657
7,146.000000,71,183.6,404.000000,40189,20848,17.8,0.000000,51.7,50.8,...,33.1,25.9,50.9,24.1,89.406636,0.305159,1.889077,2.286268,48.967033,5.889179
14,2265.000000,901,171.0,440.700000,50083,490945,16.3,462.373586,37.2,35.7,...,50.6,42.5,36.5,21.4,89.038167,1.827041,2.315986,1.033625,48.188377,5.355836
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3033,1962.667684,7,209.6,453.549422,45353,1843,11.4,0.000000,45.7,43.8,...,52.7,43.9,32.2,9.2,97.960199,0.000000,0.547264,0.995025,57.780612,4.664723
3034,1962.667684,85,184.4,453.549422,45180,35788,15.0,1900.078239,38.2,36.5,...,52.3,46.2,34.4,17.4,87.718617,3.168048,1.610355,1.893068,50.730567,7.404551
3043,1962.667684,43,150.1,453.549422,48609,37118,18.8,377.175494,30.4,29.3,...,53.3,48.6,28.8,17.7,75.706245,2.326771,4.044920,14.130288,52.007937,8.186470
3044,1962.667684,46,153.9,453.549422,51144,34536,15.0,1968.959926,30.9,30.5,...,52.6,47.8,26.6,16.8,87.961629,2.313188,1.316472,5.680705,55.153949,7.809192


In [113]:
cancer_df.shape

(591, 32)

# Removing outliers with IQR

In [114]:
Q1 = cancer_df.quantile(0.25)
Q3 = cancer_df.quantile(0.75)
IQR = Q3 - Q1

lower = Q1 - 1.5*IQR
upper = Q3 + 1.5*IQR


cancer_df = cancer_df[~((cancer_df < lower) | (cancer_df > upper)).any(axis=1)]

y = cancer_df["TARGET_deathRate"]
cancer_df = cancer_df.drop(columns="TARGET_deathRate")


In [115]:
vif_data = pd.DataFrame()
vif_data["feature"] = cancer_df.columns
vif_data["VIF"] = [variance_inflation_factor(cancer_df.values, i) for i in range(len(cancer_df.columns))]

filter = vif_data[vif_data["VIF"]>400]
filter

,feature,VIF
7,MedianAge,31114.149108
8,MedianAgeMale,8477.231368
9,MedianAgeFemale,10063.257067
10,AvgHouseholdSize,988.093614
11,PercentMarried,1253.699861
13,PctHS18_24,859.969571
14,PctSomeCol18_24,934.458142
18,PctEmployed16_Over,492.989469
20,PctPrivateCoverage,3952.360306
21,PctPrivateCoverageAlone,3417.831134


In [116]:
cancer_df = cancer_df.drop(columns=filter["feature"])

In [117]:
vif_data = pd.DataFrame()
vif_data["feature"] = cancer_df.columns
vif_data["VIF"] = [variance_inflation_factor(cancer_df.values, i) for i in range(len(cancer_df.columns))]
vif_data

,feature,VIF
0,avgAnnCount,189.799361
1,avgDeathsPerYear,157.331301
2,incidenceRate,162.468622
3,medIncome,157.311317
4,popEst2015,84.976685
5,povertyPercent,63.150487
6,studyPerCap,1.466290
7,PctNoHS18_24,10.549683
8,PctBachDeg18_24,5.516000
9,PctHS25_Over,79.449459


In [118]:
cancer_df["TARGET_deathRate"] = y

In [119]:
cancer_df.shape

(241, 17)

In [120]:
cancer_df

,avgAnnCount,avgDeathsPerYear,incidenceRate,medIncome,popEst2015,povertyPercent,studyPerCap,PctNoHS18_24,PctBachDeg18_24,PctHS25_Over,PctBachDeg25_Over,PctUnemployed16_Over,PctBlack,PctAsian,PctOtherRace,BirthRate,TARGET_deathRate
1,173.0,70,411.6,48127,43269,18.6,23.111234,6.1,7.5,26.0,22.7,7.8,0.969102,2.246233,3.741352,4.333096,161.3
4,57.0,26,350.1,49955,10321,12.5,0.000000,14.9,2.0,33.4,15.0,4.8,0.270192,0.665830,0.492135,6.796657,144.4
22,94.0,41,445.2,35615,16704,21.5,0.000000,9.8,8.3,47.1,7.9,9.0,0.836770,0.376547,0.029885,2.292861,189.7
34,250.0,103,494.9,36996,35516,21.1,0.000000,22.4,13.3,44.0,11.2,7.2,2.299108,0.386923,0.426176,4.354354,195.1
39,182.0,78,475.8,41121,29237,16.7,0.000000,13.7,9.2,41.1,11.7,6.8,0.663112,0.201668,0.061526,4.735242,194.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2963,60.0,24,465.5,49837,9410,12.1,0.000000,20.9,3.5,44.0,7.9,3.4,0.326522,0.221192,0.105330,6.748466,165.8
2968,221.0,91,466.2,52750,36827,11.2,27.153990,16.3,7.3,39.7,14.5,5.4,1.475794,0.574525,0.138866,6.996053,181.7
2976,66.0,21,475.6,50924,11185,11.1,0.000000,20.2,6.2,37.3,9.7,4.8,0.913850,2.537486,3.371484,7.112971,148.0
2981,191.0,76,471.0,57046,33294,11.1,0.000000,8.2,7.0,35.7,17.5,4.0,0.547401,1.046679,0.418070,4.504381,176.6


In [121]:
cancer_df.to_csv("..\data\cancer_reg_final.csv")